# Задание 10.

Вам предоставлен набор данных по качеству воздуха в китайских городах в виде csv файлов.
Основная метрика, которая показывает качество воздуха - PM25(<a href='https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4740125/'>подробнее о PM25</a>), указана в колонках, которые начинаются на PM_.
Остальные столбцы содержат либо данные о времени, в которое были произведены измерения, либо данные о погоде в этот момент. DEWP - точка росы, HUMI - влажность, PRES - давление, TEMP - температура, cbwd - направление ветра, Iws - его скорость, precipation - осадки, Iprec - накопленные осадки.

Для выполнения задания необходимо ответить на вопросы по данным, заполнив ячейки ноутбука. Сам ноутбук необходимо выложить в виде отдельного репозитория на github. Ссылку для клонирования репозитория необходимо приложить в решение задания.

## 1. Загрузите в виде DataFrame набор данных по качеству воздуха Пекина
Ссылка на набор данных: https://disk.yandex.ru/d/1UKJoV1ULW1NLQ

In [3]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv('BeijingPM20100101_20151231.csv')

In [9]:
data.head()

,No,year,month,day,hour,season,PM_Dongsi,PM_Dongsihuan,PM_Nongzhanguan,PM_US Post,DEWP,HUMI,PRES,TEMP,cbwd,Iws,precipitation,Iprec
0,1,2010,1,1,0,4,NaN,NaN,NaN,NaN,-21.0,43.0,1021.0,-11.0,NW,1.79,0.0,0.0
1,2,2010,1,1,1,4,NaN,NaN,NaN,NaN,-21.0,47.0,1020.0,-12.0,NW,4.92,0.0,0.0
2,3,2010,1,1,2,4,NaN,NaN,NaN,NaN,-21.0,43.0,1019.0,-11.0,NW,6.71,0.0,0.0
3,4,2010,1,1,3,4,NaN,NaN,NaN,NaN,-21.0,55.0,1019.0,-14.0,NW,9.84,0.0,0.0
4,5,2010,1,1,4,4,NaN,NaN,NaN,NaN,-20.0,51.0,1018.0,-12.0,NW,12.97,0.0,0.0


## 2. Сколько всего строк находится в наборе данных?

In [10]:
data.shape[0]

52584

## 3. Сколько измерений уровня загрязнения воздуха находится в наборе данных?

In [11]:
data.shape[0] * 4 - (data.PM_Dongsi.isna().sum() + data.PM_Dongsihuan.isna().sum() + data.PM_Nongzhanguan.isna().sum() + data['PM_US Post'].isna().sum())

120878

## 4. Введите столбец среднего качества воздуха PM_mean с усреднением показаний со всех станций слежения.

In [12]:
data['PM_Dongsi'] = data['PM_Dongsi'].fillna(data['PM_Dongsi'].mean())
data['PM_Dongsihuan'] = data['PM_Dongsihuan'].fillna(data['PM_Dongsihuan'].mean())
data['PM_Nongzhanguan'] = data['PM_Nongzhanguan'].fillna(data['PM_Nongzhanguan'].mean())
data['PM_US Post'] = data['PM_US Post'].fillna(data['PM_US Post'].mean())

In [13]:
data['PM_mean'] = range(0, data.shape[0])
for i in range(0, data.shape[0]):
  data['PM_mean'][i] = (data.PM_Dongsi[i] + data.PM_Dongsihuan[i] + data.PM_Nongzhanguan[i] + data['PM_US Post'][i])/4

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
data

,No,year,month,day,hour,season,PM_Dongsi,PM_Dongsihuan,PM_Nongzhanguan,PM_US Post,DEWP,HUMI,PRES,TEMP,cbwd,Iws,precipitation,Iprec,PM_mean
0,1,2010,1,1,0,4,89.154439,92.560806,88.643737,95.904241,-21.0,43.0,1021.0,-11.0,NW,1.79,0.0,0.0,91
1,2,2010,1,1,1,4,89.154439,92.560806,88.643737,95.904241,-21.0,47.0,1020.0,-12.0,NW,4.92,0.0,0.0,91
2,3,2010,1,1,2,4,89.154439,92.560806,88.643737,95.904241,-21.0,43.0,1019.0,-11.0,NW,6.71,0.0,0.0,91
3,4,2010,1,1,3,4,89.154439,92.560806,88.643737,95.904241,-21.0,55.0,1019.0,-14.0,NW,9.84,0.0,0.0,91
4,5,2010,1,1,4,4,89.154439,92.560806,88.643737,95.904241,-20.0,51.0,1018.0,-12.0,NW,12.97,0.0,0.0,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52579,52580,2015,12,31,19,4,140.000000,157.000000,122.000000,133.000000,-8.0,68.0,1031.0,-3.0,SE,7.14,0.0,0.0,138
52580,52581,2015,12,31,20,4,157.000000,199.000000,149.000000,169.000000,-8.0,63.0,1030.0,-2.0,SE,8.03,0.0,0.0,168
52581,52582,2015,12,31,21,4,171.000000,231.000000,196.000000,203.000000,-10.0,73.0,1030.0,-6.0,NE,0.89,0.0,0.0,200
52582,52583,2015,12,31,22,4,204.000000,242.000000,221.000000,212.000000,-10.0,73.0,1030.0,-6.0,NE,1.78,0.0,0.0,219


## 5. Какое направление ветра преобладает в Пекине?

In [15]:
data['cbwd'].value_counts().idxmax()

'SE'

## 6. В какой день качество воздуха является наихудшим(самая большая концентрация частиц)? Вывести дату в типе данных datetime стандартной библиотеки Python.

In [16]:
from datetime import date

In [17]:
max = 0
a = ''
for i in range(0, data.shape[0]):
  if data.PM_Dongsi[i] + data.PM_Dongsihuan[i] + data.PM_Nongzhanguan[i] + data['PM_US Post'][i] > max:
    max = i
print(date(day = data['day'][max], month = data['month'][max], year = data['year'][max]))

2012-01-23


## 7. Какой из сезонов в Пекине в среднем наименее неблагоприятный для легких за последние 2 года наблюдений?

In [18]:
f = 0
for i in range(0, data.shape[0]):
  if data.year[i] == 2014:
    f = i
    break
m1, m2, m3, m4 = 0, 0, 0, 0
c1, c2, c3, c4 = 0, 0, 0, 0
for i in range(f, data.shape[0]):
  if data.season[i] == 1:
    m1 += data.PM_mean[i]
    c1 += 1
  elif data.season[i] == 2:
    m2 += data.PM_mean[i]
    c2 += 1
  elif data.season[i] == 3:
    m3 += data.PM_mean[i]
    c3 += 1
  elif data.season[i] == 4:
    m4 += data.PM_mean[i]
    c4 += 1
ans1, ans2, ans3, ans4 = m1 / c1, m2 / c2, m3 / c3, m4 / c4
l = [ans1, ans2, ans3, ans4]
if np.min(l) == ans1:
  print('Season 1')
elif np.min(l) == ans2:
  print('Season 2')
elif np.min(l) == ans3:
  print('Season 3')
elif np.min(l) == ans4:
  print('Season 4')

Season 2


## 8. Коррелирует ли влажность с качеством воздуха?
Можно воспользоваться корреляцией Пирсона из numpy, а также встроенными функциями Pandas.

In [19]:
data.HUMI = data.HUMI.fillna(data.HUMI.mean())

In [20]:
np.corrcoef(data['HUMI'], data.PM_mean)

array([[1.        , 0.34502689],
       [0.34502689, 1.        ]])

## 9. Какой из сезонов в Пекине самый дождливый(в среднем по накоплению осадков)?

In [21]:
data['Iprec'] = data['Iprec'].fillna(data['Iprec'].mean())

In [22]:
m1, m2, m3, m4 = 0, 0, 0, 0
c1, c2, c3, c4 = 0, 0, 0, 0
for i in range(0, data.shape[0]):
  if data.season[i] == 1:
    m1 += data.Iprec[i]
    c1 += 1
  elif data.season[i] == 2:
    m2 += data.Iprec[i]
    c2 += 1
  elif data.season[i] == 3:
    m3 += data.Iprec[i]
    c3 += 1
  elif data.season[i] == 4:
    m4 += data.Iprec[i]
    c4 += 1
ans1, ans2, ans3, ans4 = m1 / c1, m2 / c2, m3 / c3, m4 / c4
l = [ans1, ans2, ans3, ans4]
if np.max(l) == ans1:
  print('Season 1')
elif np.max(l) == ans2:
  print('Season 2')
elif np.max(l) == ans3:
  print('Season 3')
elif np.max(l) == ans4:
  print('Season 4')

Season 3


## 10. Становится ли качество воздуха во время дождя лучше, чем оно было сразу перед дождем?

In [25]:
clear = 0
clear_count = 0
rain = 0
rain_count = 0
for i in range(0, data.shape[0]):
  if data.Iprec[i] == 0:
    clear += data.PM_mean[i]
    clear_count += 1
  else:
    rain += data.PM_mean[i]
    rain_count += 1
print('Среднее качество воздуха во время дождя: ', rain / rain_count)
print('Среднее качество воздуха перед дождём: ', clear / clear_count)

Среднее качество воздуха во время дождя:  80.11889511609287
Среднее качество воздуха перед дождём:  91.67264305394721
